In [ ]:
import os
import statistics
import math
import numpy as np

from kalman_filter import KalmanFilter
from ultra_parse import parse_csv

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
def plot_data(axis, data, start_at_time_zero=True, label_prefix=None):
    for name in sorted(data.keys()):
        plt_data = np.array(data[name])
        time_offset = plt_data[0, 0] if start_at_time_zero else 0.0
        x_vals = plt_data[:,0] - time_offset
        y_vals = plt_data[:,1]
        y_errs = None
        if len(plt_data[0]) > 2:
            y_errs = plt_data[:,2]
        if y_errs is None:
            axis.plot(x_vals, y_vals, label=name)
        else:
            #axis.errorbar(x_vals, y_vals, y_errs, label=name)
            axis.plot(x_vals, y_vals, label=name)
            y_upper = y_vals + y_errs
            y_lower = y_vals - y_errs
            axis.fill_between(x_vals, y_lower, y_upper, color='wheat')
        if label_prefix is not None:
            axis.set_title(label_prefix)
    axis.legend()

In [ ]:
a45_into_files = []
a45_from_files = []
a45_into_from_files = []
for log_file in os.listdir("logs/"):
    if log_file.startswith("hall_angle_45_into_hall_"):
        a45_into_files.append(log_file)
    elif log_file.startswith("hall_angle_45_from_hall_"):
        a45_from_files.append(log_file)
    elif log_file.startswith("hall_angle_45_into_from_hall_"):
        a45_into_from_files.append(log_file)
        
def parse_and_merge(log_file, all_data):
    data = parse_csv("logs/" + log_file)
    run_number = log_file[len("hall_angle_45_into_hall_"):-len(".csv")]
    for name, values in data.items():
        all_data[f"run_{run_number}"] = values
    return all_data

data_empty = parse_csv("logs/hall_angle_45_empty.csv")

data_into = {}
for into_file in a45_into_files:
    parse_and_merge(into_file, data_into)

data_from = {}
for from_file in a45_from_files:
    parse_and_merge(from_file, data_from)

data_into_from = {}
for if_file in a45_into_from_files:
    parse_and_merge(if_file, data_into_from)

    
#data = parse_csv("logs/hall_angle_45_into_from_hall_three_times_1.csv")

In [ ]:
def moving_average(data, window_size = 0.5):
    current_window = []
    filtered_data = []
    for d in data:
        current_window.append(d)
        while current_window and d[0] - current_window[0][0] > window_size:
            current_window.pop(0)
        all_data = sum([cd[1] for cd in current_window])
        mavg = all_data/len(current_window)
        filtered_data.append((d[0], mavg))
    return filtered_data

In [ ]:
def kalman_filter(data, min_valid_dist=0.0):
    kf = KalmanFilter()
    filtered_distance = []
    filtered_velocity = []
    filtered_cov = []
    predicted_distance = []
    predicted_velocity = []
    predicted_cov = []
    step = 0
    max_debug_steps = 0
    np.set_printoptions(precision=3, suppress=True)
    for d in data:
        if d[1] < min_valid_dist:
            continue
        step += 1
        if len(data) > 600 and step % 10000 == 0:
            print(f"At {step}/{len(data)} - {step/len(data)*100:.1f}% done")
        if not filtered_distance:
            delta_t = 0
        else:
            delta_t = d[0] - filtered_distance[-1][0]
        if step < max_debug_steps:
            print(f"\nAt Step {step} - delta_t: {delta_t}")
            print(f"State: \n{kf.state}")
            print(f"Cov: \n{kf.cov}")

        kf.predict(delta_t)
        if step < max_debug_steps:
            kf.print_predict()

        predicted_distance.append((d[0], kf.state_predict[0]))
        predicted_velocity.append((d[0], kf.state_predict[1]))
        predicted_cov.append((d[0], kf.cov_predict))

        kf.correct(d[1])
        if step < max_debug_steps:
            kf.print_correct()
        
        filtered_distance.append((d[0], kf.state[0]))
        filtered_velocity.append((d[0], kf.state[1]))
        filtered_cov.append((d[0], kf.cov))
    return filtered_distance, filtered_velocity, filtered_cov, predicted_distance, predicted_velocity, predicted_cov

In [ ]:
def linearize_cov(covs):
    return [(ts, np.linalg.eigvals(cov)) for ts, cov in covs]

In [ ]:
if __name__ == "__main__":
    run = "run_1"
    run = "run_hall_three_times_1"
    data_values = data_into_from[run]
    mavg_data = moving_average(data_values)
    kalman_dist, kalman_vel, kalman_cov, predict_dist, predict_vel, predict_cov = kalman_filter(data_values)
    lin_covs = linearize_cov(kalman_cov)
    predict_lin_covs = linearize_cov(predict_cov)
    kalman_dist_cov00 = [(ts, c[0, 0]+1) for ts, c in kalman_cov]
    kalman_dist_cov = [(ts, cov[0]) for ts, cov in lin_covs]
    kalman_vel_cov11 = [(ts, c[1,1]+1) for ts, c in kalman_cov]
    kalman_vel_cov = [(ts, cov[1]) for ts, cov in lin_covs]
    kalman_dist_errs = [(ts, d, kalman_dist_cov[i][1]) for i, (ts, d) in enumerate(kalman_dist)]
    kalman_vel_errs = [(ts, v, kalman_vel_cov[i][1]) for i, (ts, v) in enumerate(kalman_vel)]
    predict_dist_cov = [(ts, cov[0]) for ts, cov in predict_lin_covs]
    predict_vel_cov = [(ts, cov[1]) for ts, cov in predict_lin_covs]
    predict_dist_errs = [(ts, d, predict_dist_cov[i][1]) for i, (ts, d) in enumerate(predict_dist)]
    predict_vel_errs = [(ts, v, predict_vel_cov[i][1]) for i, (ts, v) in enumerate(predict_vel)]

    current_run = {run : data_into_from[run],
                   #run + "_moving_average" : mavg_data,
                   run + "_kalman_distance" : kalman_dist_errs,
                   #run + "kalman_velocity" : kalman_vel,
                  }
    current_run_vels = {#run : data_from[run],
                   #run + "_moving_average" : mavg_data,
                   #run + "_kalman_distance" : kalman_dist,
                   run + "_kalman_velocity" : kalman_vel_errs,
                  }

    predict_run = {#run : data_from[run],
                   run + "_predict_distance" : predict_dist_errs,
                  }
    predict_run_vels = {#run : data_from[run],
                   run + "_predict_velocity" : predict_vel_errs,
                  }
    current_run_covs = {#run : data_from[run],
                   #run + "_moving_average" : mavg_data,
                   run + "_kalman_dist_cov00" : kalman_dist_cov00,
                   run + "_kalman_dist_cov" : kalman_dist_cov,
                   run + "_kalman_vel_cov11" : kalman_vel_cov11,
                   run + "_kalman_vel_cov" : kalman_vel_cov,
                  }


    fig, axes = plt.subplots(2,1, figsize=(15,20))
    ylim = [0.0, 1.7]
    axes[0].set_ylim(ylim)
    #axes[2].set_ylim(ylim)
    plot_data(axes[0], current_run, start_at_time_zero=False, label_prefix="45/into_from")
    plot_data(axes[1], current_run_vels, start_at_time_zero=False, label_prefix="45/into_from")
    #plot_data(axes[2], predict_run, start_at_time_zero=False, label_prefix="45/from")
    #plot_data(axes[3], predict_run_vels, start_at_time_zero=False, label_prefix="45/from")
    #plot_data(axes[4], current_run_covs, start_at_time_zero=False, label_prefix="45/from")